In [0]:
# Required libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import lightgbm as lgb
warnings.simplefilter('ignore')
matplotlib.rcParams['figure.dpi'] = 100
sns.set()
%matplotlib inline

In [2]:
#Start by connecting gdrive into the google colab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Loading data
building = pd.read_csv('gdrive/My Drive/ASHRAE/building_metadata.csv')
weather_train = pd.read_csv('gdrive/My Drive/ASHRAE/weather_train.csv')
weather_test = pd.read_csv('gdrive/My Drive/ASHRAE/weather_test.csv')
train = pd.read_csv('gdrive/My Drive/ASHRAE/train.csv')
test = pd.read_csv('gdrive/My Drive/ASHRAE/test.csv')

In [0]:
#Merging everything into two datasets: train and test
train = train.merge(building, on='building_id', how='left')
test = test.merge(building, on='building_id', how='left')
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
gc.collect();

In [0]:
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["hour"] = train["timestamp"].dt.hour
train["day"] = train["timestamp"].dt.day
train["weekend"] = train["timestamp"].dt.weekday
train["month"] = train["timestamp"].dt.month

In [0]:
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = test["timestamp"].dt.hour
test["day"] = test["timestamp"].dt.day
test["weekend"] = test["timestamp"].dt.weekday
test["month"] = test["timestamp"].dt.month

In [0]:
train = train.drop("timestamp", axis = 1)
test = test.drop("timestamp", axis = 1)

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])

In [11]:
categoricals = ["building_id", "primary_use", "hour", "day", "weekend", "month", "meter"]
numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage","dew_temperature"]
feat_cols = categoricals + numericals
train[categoricals + numericals]

,building_id,primary_use,hour,day,weekend,month,meter,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature
0,0,0,0,1,4,1,0,7432,2008.0,25.0,6.0,20.0
1,1,0,0,1,4,1,0,2720,2004.0,25.0,6.0,20.0
2,2,0,0,1,4,1,0,5376,1991.0,25.0,6.0,20.0
3,3,0,0,1,4,1,0,23685,2002.0,25.0,6.0,20.0
4,4,0,0,1,4,1,0,116607,1975.0,25.0,6.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,1,23,31,5,12,0,19619,1914.0,1.7,NaN,-5.6
20216096,1445,0,23,31,5,12,0,4298,NaN,1.7,NaN,-5.6
20216097,1446,1,23,31,5,12,0,11265,1997.0,1.7,NaN,-5.6
20216098,1447,4,23,31,5,12,0,29775,2001.0,1.7,NaN,-5.6


In [0]:
target = np.log1p(train["meter_reading"])

In [0]:
del train["meter_reading"]

In [0]:
train = train.drop(["site_id","floor_count"], axis = 1)

In [0]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [19]:
train, NAlist = reduce_mem_usage(train)

Memory usage of properties dataframe is : 2622.022247314453  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this col:  875000
dtype after:  uint32
******************************
******************************
Column:  year_built
dtype before:  float64
min for this col:  1900.0
max for this col:  2017.0
dtype after:  uint16
******************************
******************************
Column:  air_temperature
dtype before:  float64
min for 

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
num_folds = 5
kf = KFold(n_splits = num_folds, shuffle = False, random_state = 30)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(train)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    train_X = train[feat_cols].iloc[train_index]
    val_X = train[feat_cols].iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'learning_rate': 0.05,
            'feature_fraction': 0.75,
            'bagging_fraction': 0.75,
            'bagging_freq' : 5
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0])
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.5,
            'feature_fraction': 0.75,
            'bagging_fraction': 0.75,
            'bagging_freq' : 5
            }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

16172879 16172880
Training until validation scores don't improve for 20 rounds.
[20]	training's binary_logloss: 0.213314	valid_1's binary_logloss: 0.199683
[40]	training's binary_logloss: 0.184489	valid_1's binary_logloss: 0.185121
[60]	training's binary_logloss: 0.168305	valid_1's binary_logloss: 0.180326
[80]	training's binary_logloss: 0.159696	valid_1's binary_logloss: 0.176637
[100]	training's binary_logloss: 0.153271	valid_1's binary_logloss: 0.176291
Early stopping, best iteration is:
[87]	training's binary_logloss: 0.157185	valid_1's binary_logloss: 0.174815
Training until validation scores don't improve for 20 rounds.
[20]	training's rmse: 0.857922	valid_1's rmse: 0.918223
[40]	training's rmse: 0.767992	valid_1's rmse: 0.839355
[60]	training's rmse: 0.697506	valid_1's rmse: 0.789033
[80]	training's rmse: 0.663772	valid_1's rmse: 0.772046
[100]	training's rmse: 0.632654	valid_1's rmse: 0.755337
[120]	training's rmse: 0.611014	valid_1's rmse: 0.747567
[140]	training's rmse: 0.596

In [21]:
sorted(zip(gbm_regress.feature_importance(), gbm_regress.feature_name()),reverse = True)

[(1706, 'building_id'),
 (1567, 'square_feet'),
 (725, 'meter'),
 (662, 'year_built'),
 (610, 'primary_use'),
 (464, 'month'),
 (362, 'hour'),
 (349, 'air_temperature'),
 (214, 'dew_temperature'),
 (100, 'weekend'),
 (85, 'day'),
 (26, 'cloud_coverage')]

In [0]:
import gc
del train

In [0]:
del train_X, val_X, lgb_train, lgb_eval, train_y, val_y, y_pred, target

In [24]:
gc.collect()

707

In [0]:
#test load
test["primary_use"] = le.transform(test["primary_use"])

In [26]:
test, NAlist = reduce_mem_usage(test)

Memory usage of properties dataframe is : 6362.548828125  MB
******************************
Column:  row_id
dtype before:  int64
min for this col:  0
max for this col:  41697599
dtype after:  uint32
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this c

In [27]:
gc.collect()

0

In [0]:
test = test[feat_cols]

In [29]:
from tqdm import tqdm
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    
    res.append(np.expm1((gbm_class.predict(test.iloc[i:i+step_size], num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(test.iloc[i:i+step_size], num_iteration=gbm_regress.best_iteration))))
    i+=step_size

100%|██████████| 834/834 [03:06<00:00,  4.21it/s]


In [0]:
del test

In [0]:
res = np.concatenate(res)

In [32]:
pd.DataFrame(res).describe()

,0
count,4.169760e+07
mean,7.355825e+02
std,3.344502e+04
min,-8.272226e-01
25%,2.565220e+01
50%,8.507782e+01
75%,2.575812e+02
max,1.850766e+07


In [33]:
res.shape

(41697600,)

In [0]:
sub = pd.read_csv("gdrive/My Drive/ASHRAE/sample_submission.csv")

In [0]:
sub["meter_reading"] = res

In [0]:
sub.to_csv("gdrive/My Drive/ASHRAE/submission2.csv", index = False)